In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd

# 1. 파일 로드
file_path = '1.csv'
data = pd.read_csv(file_path, encoding='CP949')

# 2. 결측치 처리
data.fillna(data.median(), inplace=True)

# 3. LSTM 모델 구현
# 타겟 변수와 특성 변수 분리
X = data.drop(['all_product'], axis=1)
y = data['all_product']

# 데이터 스케일링
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# 훈련 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# LSTM에 입력하기 위한 차원 변경
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# 모델 구성
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.01))
model.add(LSTM(50, return_sequences=True))  # LSTM 층 추가 (return_sequences=True로 설정)
model.add(Dropout(0.01))
model.add(LSTM(50, return_sequences=False))  # LSTM 층 추가 (return_sequences=False로 설정)
model.add(Dropout(0.01))
model.add(Dense(64))  # Dense 층 추가
model.add(Dense(1))  # 추가한 Dense 층

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, batch_size=10, validation_data=(X_test, y_test), verbose=1)

# 4. 성능 평가 지표 추가
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    return mse, rmse, mae, r2

# 모델 성능 평가
mse, rmse, mae, r2 = evaluate_model(model, X_test, y_test)

mse, rmse, mae, r2

Epoch 1/20
348/348 [==============================] - 13s 10ms/step - loss: 0.0126 - val_loss: 0.0116
Epoch 2/20
348/348 [==============================] - 2s 6ms/step - loss: 0.0121 - val_loss: 0.0113
Epoch 3/20
348/348 [==============================] - 2s 6ms/step - loss: 0.0121 - val_loss: 0.0111
Epoch 4/20
348/348 [==============================] - 3s 7ms/step - loss: 0.0120 - val_loss: 0.0122
Epoch 5/20
348/348 [==============================] - 4s 10ms/step - loss: 0.0118 - val_loss: 0.0109
Epoch 6/20
348/348 [==============================] - 2s 6ms/step - loss: 0.0118 - val_loss: 0.0109
Epoch 7/20
348/348 [==============================] - 2s 6ms/step - loss: 0.0118 - val_loss: 0.0108
Epoch 8/20
348/348 [==============================] - 2s 6ms/step - loss: 0.0117 - val_loss: 0.0108
Epoch 9/20
348/348 [==============================] - 2s 6ms/step - loss: 0.0117 - val_loss: 0.0109
Epoch 10/20
348/348 [==============================] - 3s 8ms/step - loss: 0.0118 - val_loss: 0.0

(0.010906524576729738,
 0.10443430746995806,
 0.06871359132734148,
 0.060615653823544724)

In [8]:
user_input = [177.45, 66, 3, 2, 12 ]

# 스케일링
user_input_scaled = scaler_X.transform([user_input])

# LSTM 모델에 맞는 차원으로 변환
user_input_scaled = user_input_scaled.reshape((1, 1, len(user_input)))

# 예측 수행
predicted_output_scaled = model.predict(user_input_scaled)

# 예측 결과 역변환
predicted_output = scaler_y.inverse_transform(predicted_output_scaled)

# 예측 결과 출력
print("예측된 all_product 값:", predicted_output[0][0])

1/1 [==============================] - 0s 24ms/step
예측된 all_product 값: 341.33035


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
